## Import libraries

In [1]:
import tweepy
import json
from pandas.io.json import json_normalize

## Setup connection

In [2]:
filename = "credentials.json"
with open(filename) as file:
    keys = json.loads(file.read())

In [3]:
consumer_key = keys['Twitter']['consumer_key']
consumer_secret = keys['Twitter']['consumer_secret']
access_token = keys['Twitter']['access_token']
access_secret = keys['Twitter']['access_secret']
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True,
                 retry_count=1000,
                 retry_delay=60)

## Download replies tweets
- [Sony live streaming](https://twitter.com/Sony/status/1214350830981148673)
- [AMD Conference announce](https://twitter.com/AMD/status/1212119569487351808)

In [4]:
def download_replies(user_name, tweet_id):
    replies = tweepy.Cursor(api.search,
                            q='to:{} -filter:retweets'.format(user_name),
                            since_id=tweet_id,
                            tweet_mode='extended',
                            lang='en').items()
    all_repl = []
    while True:
        try:
            reply = replies.next()
            all_repl.append(reply)
            if not hasattr(reply, 'in_reply_to_status_id_str'):
                continue
            if reply.in_reply_to_status_id == tweet_id:
                logging.info("reply of tweet:{}".format(reply.full_text))

        except tweepy.RateLimitError as e:
            logging.error("Twitter api rate limit reached".format(e))
            time.sleep(60)
            continue

        except tweepy.TweepError as e:
            logging.error("Tweepy error occured:{}".format(e))
            break

        except StopIteration:
            break

        except Exception as e:
            logging.error("Failed while fetching replies {}".format(e))
            break

    all_repl_processed = [0]*len(all_repl)
    for i in range(len(all_repl)):
        all_repl_processed[i] = {}
        all_repl_processed[i]["username"] = all_repl[i].user.screen_name
        all_repl_processed[i]["text"] = all_repl[i].full_text
        all_repl_processed[i]["lang"] = all_repl[i].lang
    return all_repl_processed

### Download replies to Sony tweet

In [5]:
user_name = "@Sony"
tweet_id = "1214350830981148673"
sony_tweets = download_replies(user_name=user_name, tweet_id=tweet_id)
sony_tweets = json_normalize(sony_tweets)
sony_tweets.head()

Rate limit reached. Sleeping for: 520


,username,text,lang
0,RandomWebSurf,@Sony @PlayStation will I be able to play 4K a...,en
1,jkkgold,@Sony @bts_bighit @BTS_twt you have the BEST c...,en
2,nasiradeen03,@Sony @bts_bighit @BTS_twt Anyone knew what wa...,en
3,DoItLikeMeezy,@Sony @PlayStation set themselves up for these...,en
4,bowyluv,@Sony @bts_bighit @BTS_twt bts legends \nhttps...,en


### Download replies to AMD tweet

In [6]:
user_name = "@AMD"
tweet_id = "1212119569487351808"
amd_tweets = download_replies(user_name=user_name, tweet_id=tweet_id)
amd_tweets = json_normalize(amd_tweets)
amd_tweets.head()

,username,text,lang
0,jtrathgeber,"@AMD Apparently, it wasn’t too impressive, bec...",en
1,J_M_1987_,@AMD Why comments section is disabled ? And wh...,en
2,techthelead,"@AMD has announced several new processors, lap...",en
3,bernhardtravis,@AMD @Radeon @AMDRyzen i would absolutely love...,en
4,laszlo_1337,@AMD @LisaSu @anandtech Wen ray tracing,en


### Download replies to Samsung tweet 

In [8]:
user_name = "@SamsungMobile"
tweet_id = "1159826737770856448"
samsung_tweets = download_replies(user_name=user_name, tweet_id=tweet_id)
samsung_tweets = json_normalize(samsung_tweets)
samsung_tweets.head()

,username,text,lang
0,unionjack81,@SamsungMobile any chance you could send your ...,en
1,SirEdwin_,@SamsungMobile @SamsungMobileKE can we have '...,en
2,andygray27,@SamsungMobile I'm not asking for warranty rep...,en
3,nj_ramz,@SamsungMobile Hi I tried my best in this draw...,en
4,gashead1974,@SamsungMobile is the s10 the softest screen e...,en


## Store tweets

In [9]:
sony_tweets.shape

(1779, 3)

In [10]:
amd_tweets.shape

(283, 3)

In [11]:
samsung_tweets.shape

(512, 3)

In [12]:
sony_tweets.to_csv('sony_replies.csv', index=False)
amd_tweets.to_csv('amd_replies.csv', index=False)
samsung_tweets.to_csv('samsung_tweets.csv', index=False)

## Download tweets related to CES 2020

In [19]:
def collect_tweets_from_query(api, query, number_of_tweets = 2000):

    result = []
    backoff_counter = 1
    tweets = tweepy.Cursor(api.search,
                           q="{} -filter:retweets".format(query), #for reducing the number of calls
                           lang='en'
                           ).items(number_of_tweets)

    while True:
        try:
            tweet = tweets.next()
            filtered_tweet = {
                "text": tweet.text,
                "lang": tweet.lang,
                "username": tweet.user.screen_name,
            }
            result.append(filtered_tweet)
        except StopIteration:
            break
        except tweepy.TweepError:
            print("Reached Tweet limits, waiting for {} seconds".format(60 * backoff_counter))
            sleep(60 * backoff_counter)
            backoff_counter += 1
            continue
    return result


In [20]:
ces2020_tweets = collect_tweets_from_query(api, "#CES2020")

Rate limit reached. Sleeping for: 547


In [21]:
ces2020_tweets = json_normalize(ces2020_tweets)

In [22]:
ces2020_tweets

,text,lang,username
0,Top story: @WAI_BNPP: '📣 #SaveTheDate for the ...,en,wflo
1,Attention #CES2020 goers! Be sure to hit up @H...,en,SCV_Cyber
2,That Samsung keynote presentation at CES is pr...,en,GregTric
3,Day 2 of #CES2020 will be getting underway soo...,en,AVForums
4,Are you #CleanboxClean? Stop by booth 20945 in...,en,cleanboxtech
...,...,...,...
1995,#Dell UltraSharp 43 #4k USB-C Monitor (2020) a...,en,eriktessem
1996,Huami launches Amazfit T-Rex and Blip S smartw...,en,Pricebaba
1997,Remember Google Glass? Here comes the next lev...,en,spooney35
1998,@Comp_connect My favourite was the LG's latest...,en,ShreyaPatoa


In [23]:
ces2020_tweets.to_csv("ces2020_tweets.csv", index=False)